In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as ipy

In [10]:
@ipy.interact(i=ipy.widgets.IntSlider())
def f(i):
    plt.subplot(1,2,1)
    img = plt.imread('/mnt/ml-team/dsb_2018/kuba/debug/resized{}.png'.format(i))
    plt.imshow(img)
    plt.subplot(1,2,2)
    img = plt.imread('/mnt/ml-team/dsb_2018/kuba/debug/binarized{}.png'.format(i))
    plt.imshow(img)
    plt.show()

interactive(children=(IntSlider(value=0, description='i'), Output()), _dom_classes=('widget-interact',))